In [1]:
from bs4 import BeautifulSoup
#from urllib.request import urlopen
import pandas as pd
import numpy as np
import requests

In [2]:
game_craig = pd.read_csv('games-features.csv')
game_id = game_craig.ResponseID

0            10
1            20
2            30
3            40
4            50
          ...  
13352    567660
13353    567860
13354    567940
13355    568150
13356    559250
Name: ResponseID, Length: 13357, dtype: int64

In [23]:
#This function creats a list of data frames:
def get_dfs(game_id):
    dfs = []
    fake_id = []
    for i in game_id:
        try:   
            page = requests.get("https://steamcharts.com/app/" +str(i)+ "#All").text
            soup = BeautifulSoup(page,'html')
            no_player = soup.find('table',{'class':"common-table"} )
            dfs.append(pd.read_html(str(no_player))[0])
        except:
            fake_id.append(i)
            #print("No ID number " + str(i) + "!")
    return dfs,fake_id

In [24]:
# DO NOT run this multiple times!!!

dfs = get_dfs(game_id)

#List of ResponseID's in craig's data, but not in Steam Charts
fake_id = dfs[1]

#List of ResponseID's in both craig's data and Steam Charts
real_id = [x for x in game_id if x not in fake_id]

# We make tables from Steam Charts into a list of csv files
for i in range(len(real_id)):
    dfs[0][i].to_csv('game'+str(real_id[i])+'.csv')   

In [281]:
# Make a csv file to store game IDs and names, called 'real_id.csv'.
real_id_list = [x for x in game_id if x not in fake_id]
name = [game_craig.iloc[game_craig.loc[game_craig.ResponseID==x].index[0]]['ResponseName'] for x in game_id if x not in fake_id]
real_id = pd.DataFrame({'real_id':real_id_list,'name':name})
real_id.to_csv('real_id.csv',index=False)

In [283]:
# Make csv files to store 'average number of players', 'gain', '%gain' and 'peak number of players' separately.
# DO NOT run this multiple times!!!

avg_player = []
gain = []
p_gain =[]
peak_player =[]

for i in range(len(real_id_list)):
    avg_player.append(list(dfs[0][i]['Avg. Players']) )  
    gain.append(list(dfs[0][i]['Gain']) ) 
    p_gain.append(list(dfs[0][i]['% Gain']) ) 
    peak_player.append(list(dfs[0][i]['Peak Players']) ) 
    
avg_player_df = pd.concat([real_id,pd.DataFrame(avg_player,columns = dfs[0][0].Month)],axis=1)
avg_player_df.to_csv('avg_player.csv',index=False) 

gain_df = pd.concat([real_id,pd.DataFrame(gain,columns = dfs[0][0].Month)],axis=1)
gain_df.to_csv('gain.csv',index=False) 

p_gain_df = pd.concat([real_id,pd.DataFrame(p_gain,columns = dfs[0][0].Month)],axis=1)
p_gain_df.to_csv('p_gain.csv',index=False) 

peak_player_df =pd.concat([real_id, pd.DataFrame(peak_player,columns = dfs[0][0].Month)],axis=1)
peak_player_df.to_csv('peak_player.csv',index=False) 

pd.DataFrame(fake_id).to_csv('fake_id.csv',index=False)


In [284]:
# We grab the information from the stored csv files:

real_id = pd.read_csv('real_id.csv')
fake_id = pd.read_csv('fake_id.csv')
avg_player = pd.read_csv('avg_player.csv')
gain = pd.read_csv('gain.csv')
p_gain = pd.read_csv('p_gain.csv')
peak_player = pd.read_csv('peak_player.csv')

In [285]:
avg_player

,real_id,name,Last 30 Days,April 2021,March 2021,February 2021,January 2021,December 2020,November 2020,October 2020,...,April 2013,March 2013,February 2013,January 2013,December 2012,November 2012,October 2012,September 2012,August 2012,July 2012
0,10,Counter-Strike,11721.23,12426.28,12311.34,12091.33,13094.31,12239.53,11278.70,10320.58,...,21204.46,24139.15,28378.42,34814.47,31996.07,29669.97,28836.29,29432.56,33095.80,34139.20
1,20,Team Fortress Classic,56.94,57.63,61.19,68.31,79.08,74.00,70.45,69.89,...,73.58,77.73,82.32,89.00,87.79,78.58,77.51,83.83,93.03,98.17
2,30,Day of Defeat,101.50,103.37,105.42,112.93,117.04,112.77,109.85,104.98,...,217.90,297.55,318.41,319.62,318.08,303.99,297.00,289.59,304.47,331.84
3,40,Deathmatch Classic,2.83,2.95,4.42,4.90,4.41,4.78,6.74,7.45,...,4.64,5.78,7.11,6.91,6.72,6.18,5.56,6.26,7.41,8.61
4,50,Half-Life: Opposing Force,84.70,83.77,89.22,105.72,128.28,111.27,93.25,78.46,...,43.04,39.92,43.15,50.20,47.06,38.63,41.30,56.26,55.77,79.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,565650,SWAM,0.00,0.01,0.01,0.01,0.01,0.03,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7827,565860,Tomato Way,1.68,1.71,0.84,0.64,0.65,0.62,0.19,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7828,566060,The Crypts of Anak Shaba - VR,0.45,0.67,0.23,2.15,0.32,0.69,0.60,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7829,567020,TILE,0.00,0.64,0.55,0.02,0.00,0.04,0.01,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
